In [8]:
import pandas as pd, numpy as np, statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('../processed_data/three_digit_NAICS_bds_tfp.csv')

In [ ]:
df = df.sort_values(['Industry', 'year']).copy()
df['tfp_index_bds'] = df.groupby('Industry')['tfp_index_2017'].shift(-1)
df['logTFP']        = np.log(df['tfp_index_bds'])


df['logEntry0']  = np.log(df['estabs_entry'])
df['dlogEntry0'] = df.groupby('Industry')['logEntry0'].diff()

MAX_H = 12 # forecast horizon
for h in range(1, MAX_H + 1):
    df[f'F{h}_logTFP'] = df.groupby('Industry')['logTFP'].shift(-h)

df['L0_logTFP'] = df['logTFP']
df['L1_logTFP'] = df.groupby('Industry')['logTFP'].shift(1)

irf, lo90, hi90, lo68, hi68 = [], [], [], [], []

for h in range(1, MAX_H + 1):
    dep   = f'F{h}_logTFP'
    keep  = [dep, 'dlogEntry0', 'L0_logTFP', 'L1_logTFP', 'Industry', 'year']
    dm    = df.dropna(subset=keep)

    formula = (f'{dep} ~ dlogEntry0 + L0_logTFP + L1_logTFP '
               '+ C(year) + C(Industry)')

    mod = smf.ols(formula, data=dm).fit(
            cov_type='cluster',
            cov_kwds={'groups': dm['Industry']}
          )

    beta  = mod.params['dlogEntry0']
    se    = mod.bse['dlogEntry0']

    irf.append(beta)
    hi68.append(beta + 1.00*se); lo68.append(beta - 1.00*se)
    hi90.append(beta + 1.65*se); lo90.append(beta - 1.65*se)

irf_df = pd.DataFrame({
    'horizon': range(1, MAX_H + 1),
    'IRF': irf,
    'lo68': lo68, 'hi68': hi68,
    'lo90': lo90, 'hi90': hi90
})

print(irf_df.round(4))

In [ ]:
plt.figure(figsize=(8, 5))

# 90% confidence band
plt.fill_between(
    irf_df['horizon'],
    irf_df['lo90'],
    irf_df['hi90'],
    alpha=0.2,
    label='90% CI'
)

# 68% confidence band
plt.fill_between(
    irf_df['horizon'],
    irf_df['lo68'],
    irf_df['hi68'],
    alpha=0.4,
    label='68% CI'
)

plt.plot(
    irf_df['horizon'],
    irf_df['IRF'],
    label='IRF'
)

plt.axhline(0, linestyle='--', linewidth=1)

plt.xlabel('Years After Entry Shock')
plt.ylabel('Response of Log‐TFP')
plt.title('Industry‐Level IRF of Log‐TFP to Entry Shock')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
sectors = df['Industry'].unique()  
all_irfs = []

for sec in sectors:
    dfi = df[df['Industry'] == sec].copy()

    irf, lo68, hi68, lo90, hi90 = [], [], [], [], []
    for h in range(1, 13):
        dep = f'F{h}_logTFP'
        keep = [dep, 'dlogEntry0', 'L0_logTFP', 'L1_logTFP']
        dm = dfi.dropna(subset=keep + ['year'])

        mod = smf.ols(
            f"{dep} ~ dlogEntry0 + L0_logTFP + L1_logTFP + C(year)",
            data=dm
        ).fit(
            cov_type='cluster',
            cov_kwds={'groups': dm['year']}
        )

        b, s = mod.params['dlogEntry0'], mod.bse['dlogEntry0']
        irf.append(b)
        lo68.append(b - 1 * s)
        hi68.append(b + 1 * s)
        lo90.append(b - 1.65 * s)
        hi90.append(b + 1.65 * s)

    tmp = pd.DataFrame({
        'Industry': sec,
        'horizon': range(1, 13),
        'IRF': irf,
        'lo68': lo68, 'hi68': hi68,
        'lo90': lo90, 'hi90': hi90
    })
    all_irfs.append(tmp)

irf_sector_df = pd.concat(all_irfs, ignore_index=True)

g = sns.FacetGrid(irf_sector_df,
                  col="Industry", col_wrap=4, sharey=True, height=3)
g.map_dataframe(sns.lineplot, "horizon", "IRF", color="black")
g.map_dataframe(plt.fill_between, "horizon", "lo90", "hi90",
                alpha=0.2, color="gray")
g.map_dataframe(plt.fill_between, "horizon", "lo68", "hi68",
                alpha=0.4, color="gray")
for ax in g.axes.flatten():
    ax.axhline(0, ls="--", c="red")
    ax.set_xlabel("Years After Shock")
    ax.set_ylabel("Log-TFP Response")
plt.tight_layout()
plt.show()